In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers,losses,layers
import pickle
from glob import glob
from os.path import basename
import networkx as nx
import random

In [2]:
dataset = 'LINUX' #IMDB

In [3]:
def load_generated_graphs(dataset_name, file_name='generated_graph_500'):

    dir = './dataset/' + dataset_name + '/' + file_name
    g = open(dir, 'rb')
    generated_graphs = pickle.load(g)
    g.close()
    return generated_graphs

training_pairs = load_generated_graphs(dataset, file_name='generated_graph_pairs')
test_gt_ged = load_generated_graphs(dataset, file_name='ged_matrix_test')

In [4]:
def sorted_nicely(l):
    def tryint(s):
        try:
            return int(s)
        except:
            return s

    import re
    def alphanum_key(s):
        return [tryint(c) for c in re.split('([0-9]+)', s)]

    return sorted(l, key=alphanum_key)

def load_graphs(dataset_name, train_or_test='train'):
    graphs = []
    dir = './dataset/' + dataset_name + '/' + train_or_test
    for file in sorted_nicely(glob(dir + '/*.gexf')):
        gid = int(basename(file).split('.')[0])
        g = nx.read_gexf(file)
        g.graph['gid'] = gid
        graphs.append(g)
        if not nx.is_connected(g):
            raise RuntimeError('{} not connected'.format(gid))
    return graphs

training_graphs = load_graphs(dataset, train_or_test='train')
testing_graphs = load_graphs(dataset, train_or_test='test')

In [5]:
class MyModel(tf.keras.Model):

    def __init__(self):
        super().__init__()
    
        self.input_in = tf.keras.Input(shape=(16,))
        self.fully_connected_first_in = tf.keras.layers.Dense(16, activation="relu")
        self.fully_connected_second_in = tf.keras.layers.Dense(8, activation="relu")
        self.fully_connected_third_in = tf.keras.layers.Dense(4, activation="relu")
        self.scoring_layer_in = tf.keras.layers.Dense(1, activation="sigmoid")

        self.scoring_layer_in = keras.Sequential([
            self.input_in, ##feature_count = tensor_neurons = 16
            self.fully_connected_first_in,
            self.fully_connected_second_in,
            self.fully_connected_third_in,
            self.scoring_layer_in])

        
        self.input_ie = tf.keras.Input(shape=(16,))
        self.fully_connected_first_ie = tf.keras.layers.Dense(16, activation="relu")
        self.fully_connected_second_ie = tf.keras.layers.Dense(8, activation="relu")
        self.fully_connected_third_ie = tf.keras.layers.Dense(4, activation="relu")
        self.scoring_layer_ie = tf.keras.layers.Dense(1, activation="sigmoid")

        self.scoring_layer_ie = keras.Sequential([
            self.input_ie, ##feature_count = tensor_neurons = 16
            self.fully_connected_first_ie,
            self.fully_connected_second_ie,
            self.fully_connected_third_ie,
            self.scoring_layer_ie])

        number_of_labels = 1
        self.input_dim_in = 7
        #7 for linux; 11 for IMDB 
        self.input_dim_ie = 21 
        # 21 for linux; 60 for IMDB 

        initializer1 = tf.keras.initializers.GlorotUniform()
        self.weight_matrix_in = initializer1(shape = (self.input_dim_in, self.input_dim_in, 16), dtype = tf.float32)
        self.weight_matrix_block_in = initializer1(shape= (16, 2*self.input_dim_in), dtype = tf.float32)
        self.bias_in = initializer1(shape = (16, 1), dtype = tf.float32)
        
        initializer2 = tf.keras.initializers.GlorotUniform()
        self.weight_matrix_ie = initializer2(shape = (self.input_dim_ie, self.input_dim_ie, 16), dtype = tf.float32)
        self.weight_matrix_block_ie = initializer2(shape= (16, 2*self.input_dim_ie), dtype = tf.float32)
        self.bias_ie = initializer2(shape = (16, 1), dtype = tf.float32)


    def call(self, data):
    
        adj_1 = tf.constant(np.array(data["edge_index_1"].todense()),dtype=tf.float32)
        adj_2 = tf.constant(np.array(data["edge_index_2"].todense()),dtype=tf.float32)
        features_1, features_2 = data["features_1"], data["features_2"]


        #GAL 
        graph1_hidden1 = tf.matmul(adj_1,features_1)
        graph1_hidden2 = tf.matmul(adj_1,graph1_hidden1)

        graph2_hidden1 = tf.matmul(adj_2,features_2)
        graph2_hidden2 = tf.matmul(adj_2,graph2_hidden1)
        
        
        num1 = 6
        # 10 for imdb; 6 for linux 
        num2 = 15
        # 50 for imdb; 15 for linux # the valus here can be set by the users
        Graph1_hidden1, Graph1_hidden2, Graph2_hidden1, Graph2_hidden2 = [], [], [], []
        for i in range(graph1_hidden1.shape[0]):
            if(graph1_hidden1[i][0] >= num1):
                Graph1_hidden1.append([0.0]*(num1-1) + [1.0])
            else:
                Graph1_hidden1.append([1.0 if graph1_hidden1[i][0] == j else 0.0 for j in range(num1)])

            if(graph1_hidden2[i][0] >= num2):
                Graph1_hidden2.append([0.0]*(num2-1) + [1.0])
            else:
                Graph1_hidden2.append([1.0 if graph1_hidden2[i][0] == j else 0.0 for j in range(num2)])

        for i in range(graph2_hidden1.shape[0]):
            if(graph2_hidden1[i][0] >= num1):
                Graph2_hidden1.append([0.0]*(num1-1) + [1.0])
            else:
                Graph2_hidden1.append([1.0 if graph2_hidden1[i][0] == j else 0.0 for j in range(num1)])

            if(graph2_hidden2[i][0] >= num2):
                Graph2_hidden2.append([0.0]*(num2-1) + [1.0])
            else:
                Graph2_hidden2.append([1.0 if graph2_hidden2[i][0] == j else 0.0 for j in range(num2)])
            
        Graph1_hidden1 =  tf.constant(np.array(Graph1_hidden1),dtype = tf.float32)
        Graph1_hidden2 = tf.constant(np.array(Graph1_hidden2),dtype = tf.float32)
        Graph2_hidden1 = tf.constant(np.array(Graph2_hidden1),dtype = tf.float32)
        Graph2_hidden2 = tf.constant(np.array(Graph2_hidden2),dtype = tf.float32)


        graph1_01concat = tf.concat([features_1, Graph1_hidden1], axis=1)
        graph2_01concat = tf.concat([features_2, Graph2_hidden1], axis=1)
        graph1_12concat = tf.concat([Graph1_hidden1, Graph1_hidden2], axis=1)
        graph2_12concat = tf.concat([Graph2_hidden1, Graph2_hidden2], axis=1)

        graph1_01pooled = tf.expand_dims(tf.reduce_sum(graph1_01concat,axis=0),1)
        graph1_12pooled = tf.expand_dims(tf.reduce_sum(graph1_12concat,axis=0),1)
        graph2_01pooled = tf.expand_dims(tf.reduce_sum(graph2_01concat,axis=0),1)
        graph2_12pooled = tf.expand_dims(tf.reduce_sum(graph2_12concat,axis=0),1)
        


        scoring = tf.matmul(tf.transpose(graph1_01pooled, perm=[1,0]), self.weight_matrix_in.reshape(self.input_dim_in, -1))
        scoring = scoring.reshape(self.input_dim_in, 16)
        scoring = tf.matmul(tf.transpose(scoring, perm=[1,0]), graph2_01pooled)
        combined_representation = tf.concat((graph1_01pooled, graph2_01pooled),axis=0)
        block_scoring = tf.matmul(self.weight_matrix_block_in, combined_representation)
        scores_in = tf.nn.relu(scoring + block_scoring + self.bias_in)
        scores_in = tf.transpose(scores_in, perm=[1,0])


        scores_in = self.scoring_layer_in(scores_in)
        #print("score_in: \n", scores_in)


        scoring = tf.matmul(tf.transpose(graph1_12pooled, perm=[1,0]), self.weight_matrix_ie.reshape(self.input_dim_ie, -1))
        scoring = scoring.reshape(self.input_dim_ie, 16)
        scoring = tf.matmul(tf.transpose(scoring, perm=[1,0]), graph2_12pooled)
        combined_representation = tf.concat((graph1_12pooled, graph2_12pooled),axis = 0)
        block_scoring = tf.matmul(self.weight_matrix_block_ie, combined_representation)
        scores_ie = tf.nn.relu(scoring + block_scoring + self.bias_ie)
        scores_ie = tf.transpose(scores_ie, perm=[1,0])


        scores_ie = self.scoring_layer_ie(scores_ie)
        #print("score_ie: \n", scores_ie)
        
        return(tf.concat([scores_in, scores_ie], axis=1))

In [6]:
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()
def transfer_to_tensorflow(data, type_specified=True):

        new_data = dict()
        graph1, graph2 = data['graph_pair'][0], data['graph_pair'][1]
        nodes1, nodes2 = list(graph1.nodes()), list(graph2.nodes())

        features_1, features_2 = [], []

        for n in graph1.nodes():
            features_1.append([1.0])

        for n in graph2.nodes():
            features_2.append([1.0])

        features_1, features_2 = tf.constant(np.array(features_1),dtype = tf.float32), tf.constant(np.array(features_2),dtype = tf.float32)

        new_data["edge_index_1"], new_data["edge_index_2"] = nx.adjacency_matrix(graph1), nx.adjacency_matrix(graph2)
        new_data["features_1"], new_data["features_2"] = features_1, features_2
        
        if(type_specified):
            norm_ged = [data['ged'][key] / (0.5 * (graph1.number_of_nodes() + graph2.number_of_nodes())) for key in ['in', 'ie']]
            norm_ged = np.array(norm_ged)
            new_data["target"] = tf.cast(tf.convert_to_tensor(np.exp(-norm_ged)).reshape(1, -1), tf.float32)
            
            norm_gt_ged = (data['ged']['in'] + data['ged']['ie']) / (0.5 * (graph1.number_of_nodes() + graph2.number_of_nodes()))
            new_data["gt_ged"] = tf.cast(tf.convert_to_tensor(np.exp(-norm_gt_ged).reshape(1, 1)).reshape(1, -1),tf.float32)
        else:
            norm_gt_ged = data['ged'] / (0.5 * (graph1.number_of_nodes() + graph2.number_of_nodes()))
            new_data["gt_ged"] = tf.cast(tf.convert_to_tensor(np.exp(-norm_gt_ged).reshape(1, 1)).reshape(1, -1),tf.float32)

        return new_data

In [7]:
batch_size = 128
epochs = 1
mse = tf.keras.losses.MeanSquaredError()
optimizer = keras.optimizers.Adam(learning_rate = 1e-3)
model = MyModel()

for epoch in range(epochs):
    random.shuffle(training_pairs)
    batches = []
    for graph in range(0, len(training_pairs), batch_size):
          batches.append(training_pairs[graph:graph+batch_size])
    iteration = 0
    for batch in batches:
        losses = 0  
        with tf.GradientTape() as tape:
            for graph_pair in batch:
                data = transfer_to_tensorflow(graph_pair)
                model.compile(optimizer=optimizer, loss=mse)
                test = model(data)
                losses += mse(data["target"], test)

        grads = tape.gradient(losses, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        loss = losses.numpy()    
        print ("Iteration: ", iteration, "loss: ", loss/len(batch))  
        iteration+=1
        
        
test_scores = []
for graph_1 in testing_graphs:
    for graph_2 in training_graphs:
        if((graph_1.graph['gid'], graph_2.graph['gid']) in test_gt_ged):
            curr_graph_pair = {'graph_pair': [graph_1, graph_2], 'ged':test_gt_ged[(graph_1.graph['gid'], graph_2.graph['gid'])]}
            data = transfer_to_tensorflow(curr_graph_pair, type_specified=False)
            prediction = model(data,training=False)
            prediction = tf.math.exp(tf.math.reduce_sum(tf.math.log(prediction))).reshape(1, -1)
            current_error = mse(prediction, data["gt_ged"])
            test_scores.append(current_error)


    for graph_2 in testing_graphs:
        if((graph_1.graph['gid'], graph_2.graph['gid']) in test_gt_ged):
            curr_graph_pair = {'graph_pair': [graph_1, graph_2], 'ged':test_gt_ged[(graph_1.graph['gid'], graph_2.graph['gid'])]}
            data = transfer_to_tensorflow(curr_graph_pair, type_specified=False)
            prediction = model(data,training=False)
            prediction = tf.math.exp(tf.math.reduce_sum(tf.math.log(prediction))).reshape(1, -1)
            current_error = mse(prediction, data["gt_ged"])
            test_scores.append(current_error)

model_error = sum(test_scores) / len(test_scores)
print("\nModel test error: " + str(model_error))

E:\anaconda\lib\site-packages\keras\initializers\initializers_v2.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(
C:\Users\admin\AppData\Local\Temp\ipykernel_3452\1718527493.py:19: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  new_data["edge_index_1"], new_data["edge_index_2"] = nx.adjacency_matrix(graph1), nx.adjacency_matrix(graph2)


Iteration:  0 loss:  0.050879593938589096
Iteration:  1 loss:  0.05266804248094559
Iteration:  2 loss:  0.04678270220756531
Iteration:  3 loss:  0.04492776095867157
Iteration:  4 loss:  0.04224985092878342
Iteration:  5 loss:  0.04225000739097595
Iteration:  6 loss:  0.03797530010342598
Iteration:  7 loss:  0.03466402739286423
Iteration:  8 loss:  0.03101308085024357
Iteration:  9 loss:  0.03365826606750488
Iteration:  10 loss:  0.031239930540323257
Iteration:  11 loss:  0.02659858763217926
Iteration:  12 loss:  0.026133570820093155
Iteration:  13 loss:  0.02496526762843132
Iteration:  14 loss:  0.02309233881533146
Iteration:  15 loss:  0.022282296791672707
Iteration:  16 loss:  0.021911371499300003
Iteration:  17 loss:  0.019309692084789276
Iteration:  18 loss:  0.019525950774550438
Iteration:  19 loss:  0.016732100397348404
Iteration:  20 loss:  0.017771419137716293
Iteration:  21 loss:  0.016851626336574554
Iteration:  22 loss:  0.015085771679878235
Iteration:  23 loss:  0.015670515

Iteration:  190 loss:  0.010216400027275085
Iteration:  191 loss:  0.011198168620467186
Iteration:  192 loss:  0.011876272968947887
Iteration:  193 loss:  0.010822935961186886
Iteration:  194 loss:  0.00967555120587349
Iteration:  195 loss:  0.008297594264149666
Iteration:  196 loss:  0.009683316573500633
Iteration:  197 loss:  0.00956717785447836
Iteration:  198 loss:  0.010070160031318665
Iteration:  199 loss:  0.008437033742666245
Iteration:  200 loss:  0.009044298902153969
Iteration:  201 loss:  0.010573338717222214
Iteration:  202 loss:  0.010932454839348793
Iteration:  203 loss:  0.010002264752984047
Iteration:  204 loss:  0.00998623389750719
Iteration:  205 loss:  0.00934547744691372
Iteration:  206 loss:  0.008628180250525475
Iteration:  207 loss:  0.008488434366881847
Iteration:  208 loss:  0.010007149539887905
Iteration:  209 loss:  0.008812532760202885
Iteration:  210 loss:  0.009005721658468246
Iteration:  211 loss:  0.00977679155766964
Iteration:  212 loss:  0.009536118246

Iteration:  378 loss:  0.008777568116784096
Iteration:  379 loss:  0.008810151368379593
Iteration:  380 loss:  0.010443794541060925
Iteration:  381 loss:  0.010619436390697956
Iteration:  382 loss:  0.009829402901232243
Iteration:  383 loss:  0.011865349486470222
Iteration:  384 loss:  0.01108311116695404
Iteration:  385 loss:  0.008833133615553379
Iteration:  386 loss:  0.009696369990706444
Iteration:  387 loss:  0.009693455882370472
Iteration:  388 loss:  0.01087120920419693
Iteration:  389 loss:  0.009953540749847889
Iteration:  390 loss:  0.007792054675519466
Iteration:  391 loss:  0.008346699178218842
Iteration:  392 loss:  0.009762667119503021
Iteration:  393 loss:  0.009673308581113815
Iteration:  394 loss:  0.009653330780565739
Iteration:  395 loss:  0.008609151467680931
Iteration:  396 loss:  0.010459914803504944
Iteration:  397 loss:  0.008934197947382927
Iteration:  398 loss:  0.009153950028121471
Iteration:  399 loss:  0.009496423415839672
Iteration:  400 loss:  0.009582836

Iteration:  566 loss:  0.009833495132625103
Iteration:  567 loss:  0.00930607970803976
Iteration:  568 loss:  0.007884564809501171
Iteration:  569 loss:  0.00975035410374403
Iteration:  570 loss:  0.00957983173429966
Iteration:  571 loss:  0.008154584094882011
Iteration:  572 loss:  0.009697633795440197
Iteration:  573 loss:  0.009641336277127266
Iteration:  574 loss:  0.008457871153950691
Iteration:  575 loss:  0.00951283611357212
Iteration:  576 loss:  0.008548536337912083
Iteration:  577 loss:  0.008118250407278538
Iteration:  578 loss:  0.009074181318283081
Iteration:  579 loss:  0.008505133911967278
Iteration:  580 loss:  0.00828248169273138
Iteration:  581 loss:  0.008292042650282383
Iteration:  582 loss:  0.010114668868482113
Iteration:  583 loss:  0.010167942382395267
Iteration:  584 loss:  0.008842848241329193
Iteration:  585 loss:  0.01045943982899189
Iteration:  586 loss:  0.008133133873343468
Iteration:  587 loss:  0.010077976621687412
Iteration:  588 loss:  0.0082796318456

...

Iteration:  4232 loss:  0.0077498420141637325
Iteration:  4233 loss:  0.006663117092102766
Iteration:  4234 loss:  0.007568650878965855
Iteration:  4235 loss:  0.00625265808776021
Iteration:  4236 loss:  0.007919294759631157
Iteration:  4237 loss:  0.00683053070679307
Iteration:  4238 loss:  0.00844173226505518
Iteration:  4239 loss:  0.007069369312375784
Iteration:  4240 loss:  0.007257483899593353
Iteration:  4241 loss:  0.006594038102775812
Iteration:  4242 loss:  0.007589240092784166
Iteration:  4243 loss:  0.0077463798224925995
Iteration:  4244 loss:  0.006477983668446541
Iteration:  4245 loss:  0.007959720678627491
Iteration:  4246 loss:  0.0075042974203825
Iteration:  4247 loss:  0.006971275433897972
Iteration:  4248 loss:  0.0066427080892026424
Iteration:  4249 loss:  0.007275628857314587
Iteration:  4250 loss:  0.00663399463519454
Iteration:  4251 loss:  0.006785854231566191
Iteration:  4252 loss:  0.007394743151962757
Iteration:  4253 loss:  0.00586260249838233
Iteration:  42

Iteration:  4415 loss:  0.008012840524315834
Iteration:  4416 loss:  0.006815827451646328
Iteration:  4417 loss:  0.007269225548952818
Iteration:  4418 loss:  0.006487796548753977
Iteration:  4419 loss:  0.0067906794138252735
Iteration:  4420 loss:  0.006936795078217983
Iteration:  4421 loss:  0.00856717023998499
Iteration:  4422 loss:  0.006472355220466852
Iteration:  4423 loss:  0.00695744389668107
Iteration:  4424 loss:  0.008039113134145737
Iteration:  4425 loss:  0.007952455431222916
Iteration:  4426 loss:  0.00713758310303092
Iteration:  4427 loss:  0.006246994715183973
Iteration:  4428 loss:  0.006723994389176369
Iteration:  4429 loss:  0.007647260557860136
Iteration:  4430 loss:  0.00789475068449974
Iteration:  4431 loss:  0.006271948106586933
Iteration:  4432 loss:  0.006798608228564262
Iteration:  4433 loss:  0.007561954669654369
Iteration:  4434 loss:  0.0063924421556293964
Iteration:  4435 loss:  0.00660625658929348
Iteration:  4436 loss:  0.006893334910273552
Iteration:  4

Iteration:  4597 loss:  0.007359278853982687
Iteration:  4598 loss:  0.00642347289249301
Iteration:  4599 loss:  0.007343437988311052
Iteration:  4600 loss:  0.006598540581762791
Iteration:  4601 loss:  0.005977225489914417
Iteration:  4602 loss:  0.007963740266859531
Iteration:  4603 loss:  0.006760649848729372
Iteration:  4604 loss:  0.006046324968338013
Iteration:  4605 loss:  0.006710075307637453
Iteration:  4606 loss:  0.006665152497589588
Iteration:  4607 loss:  0.0062521714717149734
Iteration:  4608 loss:  0.006584906950592995
Iteration:  4609 loss:  0.007606619969010353
Iteration:  4610 loss:  0.006684953812509775
Iteration:  4611 loss:  0.006507521029561758
Iteration:  4612 loss:  0.007085008081048727
Iteration:  4613 loss:  0.005705660209059715
Iteration:  4614 loss:  0.007674563210457563
Iteration:  4615 loss:  0.007392560597509146
Iteration:  4616 loss:  0.006402998697012663
Iteration:  4617 loss:  0.006835630629211664
Iteration:  4618 loss:  0.00772117218002677
Iteration: 

Iteration:  4780 loss:  0.007642541080713272
Iteration:  4781 loss:  0.007205108180642128
Iteration:  4782 loss:  0.006217633374035358
Iteration:  4783 loss:  0.006553285289555788
Iteration:  4784 loss:  0.008113272488117218
Iteration:  4785 loss:  0.006273087114095688
Iteration:  4786 loss:  0.007377051282674074
Iteration:  4787 loss:  0.006428133696317673
Iteration:  4788 loss:  0.0072452472522854805
Iteration:  4789 loss:  0.007086286786943674
Iteration:  4790 loss:  0.007859272882342339
Iteration:  4791 loss:  0.006715215276926756
Iteration:  4792 loss:  0.006864784751087427
Iteration:  4793 loss:  0.006985447835177183
Iteration:  4794 loss:  0.007116769906133413
Iteration:  4795 loss:  0.006801115348935127
Iteration:  4796 loss:  0.007047789636999369
Iteration:  4797 loss:  0.006525164004415274
Iteration:  4798 loss:  0.006480451673269272
Iteration:  4799 loss:  0.006821501534432173
Iteration:  4800 loss:  0.0065121129155159
Iteration:  4801 loss:  0.006279440596699715
Iteration: 

Iteration:  4962 loss:  0.006297367624938488
Iteration:  4963 loss:  0.006360162515193224
Iteration:  4964 loss:  0.007635343354195356
Iteration:  4965 loss:  0.007008796092122793
Iteration:  4966 loss:  0.006763677578419447
Iteration:  4967 loss:  0.0062549724243581295
Iteration:  4968 loss:  0.006301093380898237
Iteration:  4969 loss:  0.0060198623687028885
Iteration:  4970 loss:  0.006179117131978273
Iteration:  4971 loss:  0.00542652839794755
Iteration:  4972 loss:  0.0077054803259670734
Iteration:  4973 loss:  0.0073632001876831055
Iteration:  4974 loss:  0.006582255475223064
Iteration:  4975 loss:  0.006862555630505085
Iteration:  4976 loss:  0.006028494331985712
Iteration:  4977 loss:  0.007146912626922131
Iteration:  4978 loss:  0.007478478364646435
Iteration:  4979 loss:  0.0066936807706952095
Iteration:  4980 loss:  0.006723325699567795
Iteration:  4981 loss:  0.006967987399548292
Iteration:  4982 loss:  0.006663760170340538
Iteration:  4983 loss:  0.00646541453897953
Iterati